In [ ]:
@nb.njit
def deleteFromArray(array, obj):
    newArray = np.array((0,0))
    for coor in array:
        if coor[0] != obj[0] and coor[1] != obj[1]:
            newArray = np.append(newArray, [np.array([y0, x0])], axis=0)
    return newArray

@jit(nopython=True)
def ifInArray(coor, arr):
    inArray = False
    for pos in arr:
        if pos[0] == coor[0] and pos[1] == coor[1]:
            inArray = True
    return inArray

@jit(nopython=True)
def getIndex(nonZero, coor):
    index = 0
    for pos in nonZero:
        if pos[0] == coor[0] and pos[1] == coor[1]:
            return int(index)
        index += 1

@jit(nopython=True)
def cluster(grid, N):
    
    clusterGrid = np.zeros((N,N))

    monomersLeft = np.argwhere(grid != 0)

    monomersNext = np.array([[0,0]])
    
    val = 1
    first = True
    while monomersLeft > 0:
        
        y0, x0 = monomersLeft[0]
        monomersLeft = np.delete(monomersLeft, getIndex(monomersLeft, np.array([y0, x0])))
        monomersNext = np.append(monomersNext, [np.array([y0, x0])], axis=0)
        if first == True:
            index = int(getIndex(monomersNext, np.array([0, 0])))
            monomersNext = np.delete(monomersNext, index, axis=0)
            first = False

        size = 0
    
        while monomersNext > 0:
            
            y, x = monomersNext[0]
            clusterGrid[y,x] = val
            size += 1
            monomersNext = np.delete(monomersNext, getIndex(monomersNext, np.array([y, x])))

            right = np.array([y, (x+1)%N])
            if ifInArray(right, monomersLeft):
                monomersLeft = np.delete(monomersLeft, getIndex(monomersLeft, right))
                monomersNext = np.append(monomersNext, np.array([right]), axis=0)
            
            left = np.array([y, (x-1)%N])
            if ifInArray(left, monomersLeft):
                monomersLeft = np.delete(monomersLeft, getIndex(monomersLeft, left))
                monomersNext = np.append(monomersNext, np.array([left]), axis=0)
            
            under = np.array([(y+1)%N, x])
            if ifInArray(under, monomersLeft):
                monomersLeft = np.delete(monomersLeft, getIndex(monomersLeft, under))
                monomersNext = np.append(monomersNext, np.array([under]), axis=0)
            
            above = np.array([(y-1)%N, x])
            if ifInArray(above, monomersLeft):
                monomersLeft = np.delete(monomersLeft, getIndex(monomersLeft, above))
                monomersNext = np.append(monomersNext, np.array([above]), axis=0)
            
        val += 1

    return clusterGrid

plot = True
if plot == True:
    T = 20
    clusterGrid_200 = cluster(grid_200, 15)
    grids = [grid_200, clusterGrid_200]
    info = [["Initial", "Cluster grid"],[0, N_s],[M, M],[T2, T2],[0, 0],[False, True],[False, False]]
    plotSystems(grids, info, f"System after clustering after {N_s} steps with T = {T}")

    T2 = 500
    clusterGrid_500 = cluster(grid_500, 15)
    grids = [grid_500, clusterGrid_500]
    info = [["Initial", "Cluster grid"],[0, N_s],[M, M],[T2, T2],[0, 0],[False, True],[False, False]]
    plotSystems(grids, info, f"System after clustering after {N_s} steps with T = {T2}")
